# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np 
import math 
import requests 
from scipy.stats import percentileofscore as plscore
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secretsofalgotr import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/stat?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2341716835398,
 'week52high': 181.93,
 'week52low': 133.19,
 'week52highSplitAdjustOnly': 184.13,
 'week52lowSplitAdjustOnly': 134.5,
 'week52change': -0.0958647364399738,
 'sharesOutstanding': 16642169347,
 'float': 0,
 'avg10Volume': 56774689,
 'avg30Volume': 81566236,
 'day200MovingAvg': 152.06,
 'day50MovingAvg': 145.86,
 'employees': 151996,
 'ttmEPS': 6.28,
 'ttmDividendRate': 0.9409274148971549,
 'dividendYield': 0.006394263709880183,
 'nextDividendDate': '',
 'exDividendDate': '2022-10-23',
 'nextEarningsDate': '2023-01-23',
 'peRatio': 23.11039617492922,
 'beta': 1.35436422126865,
 'maxChangePercent': 57.234562352423104,
 'year5ChangePercent': 2.607646052013517,
 'year2ChangePercent': 0.2603604963132515,
 'year1ChangePercent': -0.07564207688284484,
 'ytdChangePercent': -0.1855538366770896,
 'month6ChangePercent': -0.03464935668714169,
 'month3ChangePercent': -0.11998633775015669,
 'month1ChangePercent': -0.07550837080560105,
 'day30Ch

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

-0.07564207688284484

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            row = pd.DataFrame([symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'],index=my_columns).T

            final_dataframe = pd.concat([final_dataframe, row], 
                                        ignore_index=True,axis=0)
        except:
            print("one err")
final_dataframe

one err
one err
one err
one err


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,152.46,0.008266,N/A
1,AAL,14.5,-0.227117,N/A
2,AAP,151.27,-0.329074,N/A
3,AAPL,144.32,-0.078112,N/A
4,ABBV,161.4,0.425613,N/A
...,...,...,...,...
496,YUM,130.62,0.038523,N/A
497,ZBH,123,-0.058143,N/A
498,ZBRA,279.64,-0.567765,N/A
499,ZION,51.51,-0.203185,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return',
                            ascending=False,inplace=True)
final_dataframe=final_dataframe[:50].reset_index().drop(['index'], axis=1)

In [8]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,81.55,2.367843,N/A
1,OXY,72.5,1.341286,N/A
2,FTI,12.45,1.088074,N/A
3,VLO,139.2,1.08581,N/A
4,MPC,125.62,1.074321,N/A
5,MRO,31.41,0.941039,N/A
6,XOM,114.63,0.898071,N/A
7,HES,143.7,0.860267,N/A
8,HRB,44.1,0.825892,N/A
9,COP,129.02,0.824756,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size
    while(True):
        portfolio_size=input('Enter the size of your portfolio:')
        
        try:
            portfolio_size=float(portfolio_size)
            break
        except:
            print("Enter only numbers :")
            continue
portfolio_input()

Enter the size of your portfolio:1000000


In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy']=math.floor(position_size/final_dataframe.loc[i,'Price'])

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,81.55,2.367843,245
1,OXY,72.5,1.341286,275
2,FTI,12.45,1.088074,1606
3,VLO,139.2,1.08581,143
4,MPC,125.62,1.074321,159
5,MRO,31.41,0.941039,636
6,XOM,114.63,0.898071,174
7,HES,143.7,0.860267,139
8,HRB,44.1,0.825892,453
9,COP,129.02,0.824756,155


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [11]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile', 
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM score'
    
]
hqm_dataframe = pd.DataFrame(columns=hqm_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        try:
            row = pd.DataFrame([symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A', 
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A',
                                
                ],index=hqm_columns).T

            hqm_dataframe = pd.concat([hqm_dataframe, row], 
                                        ignore_index=True,axis=0)
        except:
            print("one err")
hqm_dataframe


one err
one err
one err
one err


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM score
0,A,151.28,N/A,0.007963,N/A,0.173222,N/A,0.180464,N/A,0.10161,N/A,N/A
1,AAL,14.28,N/A,-0.226149,N/A,-0.248055,N/A,0.006788,N/A,-0.01263,N/A,N/A
2,AAP,154.15,N/A,-0.330743,N/A,-0.209091,N/A,-0.128428,N/A,-0.212642,N/A,N/A
3,AAPL,144.87,N/A,-0.075642,N/A,-0.033889,N/A,-0.12064,N/A,-0.074717,N/A,N/A
4,ABBV,161.9,N/A,0.432951,N/A,0.079556,N/A,0.174441,N/A,0.076543,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.59,N/A,0.038788,N/A,0.066784,N/A,0.137836,N/A,0.077654,N/A,N/A
497,ZBH,120,N/A,-0.05727,N/A,-0.013838,N/A,0.09707,N/A,0.038702,N/A,N/A
498,ZBRA,268.12,N/A,-0.561561,N/A,-0.221205,N/A,-0.129525,N/A,-0.06753,N/A,N/A
499,ZION,52.66,N/A,-0.202498,N/A,-0.110491,N/A,-0.105744,N/A,-0.018012,N/A,N/A


In [12]:
print(hqm_dataframe.isna().sum())
print(hqm_dataframe.dropna(axis=0).isna().sum())

hqm_dataframe = hqm_dataframe.dropna(axis=0)

Ticker                           0
Price                            0
Number of Shares to Buy          0
One-Year Price Return            4
One-Year Return Percentile       0
Six-Month Price Return           4
Six-Month Return Percentile      0
Three-Month Price Return         4
Three-Month Return Percentile    0
One-Month Price Return           4
One-Month Return Percentile      0
HQM score                        0
dtype: int64
Ticker                           0
Price                            0
Number of Shares to Buy          0
One-Year Price Return            0
One-Year Return Percentile       0
Six-Month Price Return           0
Six-Month Return Percentile      0
Three-Month Price Return         0
Three-Month Return Percentile    0
One-Month Price Return           0
One-Month Return Percentile      0
HQM score                        0
dtype: int64


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [13]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM score
0,A,151.28,N/A,0.007963,N/A,0.173222,N/A,0.180464,N/A,0.10161,N/A,N/A
1,AAL,14.28,N/A,-0.226149,N/A,-0.248055,N/A,0.006788,N/A,-0.01263,N/A,N/A
2,AAP,154.15,N/A,-0.330743,N/A,-0.209091,N/A,-0.128428,N/A,-0.212642,N/A,N/A
3,AAPL,144.87,N/A,-0.075642,N/A,-0.033889,N/A,-0.12064,N/A,-0.074717,N/A,N/A
4,ABBV,161.9,N/A,0.432951,N/A,0.079556,N/A,0.174441,N/A,0.076543,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.59,N/A,0.038788,N/A,0.066784,N/A,0.137836,N/A,0.077654,N/A,N/A
497,ZBH,120,N/A,-0.05727,N/A,-0.013838,N/A,0.09707,N/A,0.038702,N/A,N/A
498,ZBRA,268.12,N/A,-0.561561,N/A,-0.221205,N/A,-0.129525,N/A,-0.06753,N/A,N/A
499,ZION,52.66,N/A,-0.202498,N/A,-0.110491,N/A,-0.105744,N/A,-0.018012,N/A,N/A


In [14]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = plscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

/home/harvy/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/tmp/ipykernel_12215/2353316234.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = plscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
/home/harvy/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/tmp/ipykernel_12215/2353316234.py:10: SettingWithCopyWarning: 
A value 

/home/harvy/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/tmp/ipykernel_12215/2353316234.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = plscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
/home/harvy/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/tmp/ipykernel_12215/2353316234.py:10: SettingWithCopyWarning: 
A value 

0      0.607646
1        0.2334
2      0.140845
3      0.472837
4      0.927565
         ...   
496    0.653924
497    0.498994
498    0.018109
499     0.27163
500    0.142857
Name: One-Year Return Percentile, Length: 497, dtype: object
0      0.891348
1      0.084507
2      0.126761
3      0.452716
4      0.744467
         ...   
496    0.732394
497    0.498994
498    0.122736
499    0.289738
500    0.209256
Name: Six-Month Return Percentile, Length: 497, dtype: object
0      0.915493
1      0.468813
2      0.126761
3      0.132797
4      0.905433
         ...   
496    0.841046
497    0.752515
498    0.122736
499    0.160966
500     0.22334
Name: Three-Month Return Percentile, Length: 497, dtype: object
0      0.867203
1      0.183099
2      0.008048
3      0.052314
4      0.750503
         ...   
496      0.7666
497    0.498994
498     0.06841
499    0.158954
500    0.126761
Name: One-Month Return Percentile, Length: 497, dtype: object


/home/harvy/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/tmp/ipykernel_12215/2353316234.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = plscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
/home/harvy/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/tmp/ipykernel_12215/2353316234.py:10: SettingWithCopyWarning: 
A value 

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM score
0,A,151.28,N/A,0.007963,0.607646,0.173222,0.891348,0.180464,0.915493,0.10161,0.867203,N/A
1,AAL,14.28,N/A,-0.226149,0.2334,-0.248055,0.084507,0.006788,0.468813,-0.01263,0.183099,N/A
2,AAP,154.15,N/A,-0.330743,0.140845,-0.209091,0.126761,-0.128428,0.126761,-0.212642,0.008048,N/A
3,AAPL,144.87,N/A,-0.075642,0.472837,-0.033889,0.452716,-0.12064,0.132797,-0.074717,0.052314,N/A
4,ABBV,161.9,N/A,0.432951,0.927565,0.079556,0.744467,0.174441,0.905433,0.076543,0.750503,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.59,N/A,0.038788,0.653924,0.066784,0.732394,0.137836,0.841046,0.077654,0.7666,N/A
497,ZBH,120,N/A,-0.05727,0.498994,-0.013838,0.498994,0.09707,0.752515,0.038702,0.498994,N/A
498,ZBRA,268.12,N/A,-0.561561,0.018109,-0.221205,0.122736,-0.129525,0.122736,-0.06753,0.06841,N/A
499,ZION,52.66,N/A,-0.202498,0.27163,-0.110491,0.289738,-0.105744,0.160966,-0.018012,0.158954,N/A


In [15]:
hqm_dataframe.sample(10)

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM score
157,EMN,86.92,N/A,-0.221858,0.237425,-0.222843,0.118712,-0.105129,0.16499,0.113583,0.895372,N/A
230,HUM,545.96,N/A,0.259024,0.861167,0.164986,0.879276,0.100839,0.762575,-0.037067,0.118712,N/A
473,WAB,101.3,N/A,0.072676,0.690141,0.047173,0.678068,0.096072,0.748491,0.061601,0.653924,N/A
95,CINF,111.97,N/A,-0.062741,0.484909,-0.158384,0.185111,0.074225,0.68008,0.050154,0.589537,N/A
288,LVS,45.06,N/A,0.154416,0.778672,0.267408,0.961771,0.151022,0.863179,0.178906,0.965795,N/A
124,CXO,67.1,N/A,-0.2644,0.193159,0.31496,0.975855,0.370859,0.987928,0.105214,0.875252,N/A
202,GM,40.91,N/A,-0.358178,0.110664,0.023611,0.61167,0.006296,0.464789,0.014141,0.334004,N/A
289,LW,85.63,N/A,0.659705,0.959759,0.264563,0.957746,0.061878,0.645875,0.002939,0.285714,N/A
347,NWSA,19.44,N/A,-0.14524,0.368209,0.035064,0.645875,0.056457,0.631791,0.076401,0.748491,N/A
35,AMT,225.86,N/A,-0.168479,0.323944,-0.170259,0.16499,-0.176394,0.064386,0.045546,0.557344,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [16]:
from statistics import mean 

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM score'] = mean(momentum_percentiles)
hqm_dataframe

/tmp/ipykernel_12215/2030173023.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[row,'HQM score'] = mean(momentum_percentiles)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM score
0,A,151.28,N/A,0.007963,0.607646,0.173222,0.891348,0.180464,0.915493,0.10161,0.867203,0.820423
1,AAL,14.28,N/A,-0.226149,0.2334,-0.248055,0.084507,0.006788,0.468813,-0.01263,0.183099,0.242455
2,AAP,154.15,N/A,-0.330743,0.140845,-0.209091,0.126761,-0.128428,0.126761,-0.212642,0.008048,0.100604
3,AAPL,144.87,N/A,-0.075642,0.472837,-0.033889,0.452716,-0.12064,0.132797,-0.074717,0.052314,0.277666
4,ABBV,161.9,N/A,0.432951,0.927565,0.079556,0.744467,0.174441,0.905433,0.076543,0.750503,0.831992
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.59,N/A,0.038788,0.653924,0.066784,0.732394,0.137836,0.841046,0.077654,0.7666,0.748491
497,ZBH,120,N/A,-0.05727,0.498994,-0.013838,0.498994,0.09707,0.752515,0.038702,0.498994,0.562374
498,ZBRA,268.12,N/A,-0.561561,0.018109,-0.221205,0.122736,-0.129525,0.122736,-0.06753,0.06841,0.082998
499,ZION,52.66,N/A,-0.202498,0.27163,-0.110491,0.289738,-0.105744,0.160966,-0.018012,0.158954,0.220322


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [17]:
hqm_top_50 = hqm_dataframe.sort_values('HQM score',ascending=False)
hqm_top_50 = hqm_top_50[:50].reset_index(drop=True)

hqm_top_50

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM score
0,FTI,12.39,N/A,1.079551,0.993964,0.41442,0.98994,0.419058,0.993964,0.192605,0.973843,0.987928
1,ABMD,388.65,N/A,0.16487,0.78672,0.421485,0.991952,0.473158,0.995976,0.468053,1.0,0.943662
2,COG,23,N/A,0.342431,0.903421,0.187043,0.903421,0.262404,0.971831,0.258769,0.987928,0.94165
3,LB,81.43,N/A,2.303256,1.0,0.831924,1.0,0.2257,0.959759,0.082493,0.792757,0.938129
4,ROST,122.2,N/A,0.074122,0.696177,0.35736,0.985915,0.339806,0.983903,0.226842,0.983903,0.912475
5,MPC,127.87,N/A,1.052791,0.991952,0.218716,0.927565,0.178594,0.913481,0.080985,0.788732,0.905433
6,TJX,81.32,N/A,0.181222,0.798793,0.259374,0.949698,0.279745,0.977867,0.110301,0.89336,0.90493
7,GILD,88.84,N/A,0.283681,0.871227,0.354482,0.983903,0.390983,0.98994,0.0776,0.764588,0.902414
8,DE,447.12,N/A,0.249976,0.857143,0.232,0.937626,0.168701,0.895372,0.115683,0.901408,0.897887
9,AFL,73.08,N/A,0.314315,0.885312,0.192023,0.907445,0.174885,0.907445,0.105716,0.8833,0.895875


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [18]:
portfolio_input()

Enter the size of your portfolio:1000000


In [19]:
position_size = float(portfolio_size)/len(hqm_top_50.index)
for i in hqm_top_50.index:
    hqm_top_50.loc[i,'Number of Shares to Buy']= math.floor(position_size/hqm_top_50.loc[i,'Price'])

hqm_top_50.sample(10)

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM score
18,ABBV,161.9,123,0.432951,0.927565,0.079556,0.744467,0.174441,0.905433,0.076543,0.750503,0.831992
28,ANET,137.82,145,0.072283,0.686117,0.269436,0.963783,0.088617,0.726358,0.10269,0.871227,0.811871
9,AFL,73.08,273,0.314315,0.885312,0.192023,0.907445,0.174885,0.907445,0.105716,0.8833,0.895875
39,LIN,341.21,58,0.07247,0.688129,0.033212,0.643863,0.188514,0.937626,0.120641,0.911469,0.795272
10,LVS,45.06,443,0.154416,0.778672,0.267408,0.961771,0.151022,0.863179,0.178906,0.965795,0.892354
2,COG,23,869,0.342431,0.903421,0.187043,0.903421,0.262404,0.971831,0.258769,0.987928,0.94165
21,WMT,154.42,129,0.078219,0.704225,0.200838,0.919517,0.170307,0.899396,0.080331,0.782696,0.826459
17,MNST,102.28,195,0.176958,0.794769,0.138807,0.855131,0.144058,0.849095,0.092896,0.83501,0.833501
8,DE,447.12,44,0.249976,0.857143,0.232,0.937626,0.168701,0.895372,0.115683,0.901408,0.897887
16,WYNN,81.49,245,-0.089108,0.444668,0.203458,0.923541,0.254625,0.969819,0.349286,0.997988,0.834004


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [24]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_top_50.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [25]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [26]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [27]:
writer.save()

/tmp/ipykernel_12215/934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
